Installation

In [ ]:
! pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

LangChain supports many different language models, we can try openai gpt model with OPENAI_API_KEY

In [ ]:
! pip install -qU langchain-openai

In [6]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [2]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

Define Tools: We need to create the tools we want to use. Our main tool of choice will be Tavily - a search engine. We have a built-in tool in LangChain to easily use Tavily search engine as tool.

In [3]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter API key for TAVILY: ")

from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1737474415, 'localtime': '2025-01-21 07:46'}, 'current': {'last_updated_epoch': 1737474300, 'last_updated': '2025-01-21 07:45', 'temp_c': 5.6, 'temp_f': 42.1, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 6.3, 'wind_kph': 10.1, 'wind_degree': 51, 'wind_dir': 'NE', 'pressure_mb': 1033.0, 'pressure_in': 30.5, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 70, 'cloud': 0, 'feelslike_c': 3.4, 'feelslike_f': 38.0, 'windchill_c': 6.4, 'windchill_f': 43.5, 'heatindex_c': 7.9, 'heatindex_f': 46.2, 'dewpoint_c': 4.5, 'dewpoint_f': 40.0, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 11.5, 'gust_kph': 18.4}}"}, {'url': 'https://world-weather.info/forecast/usa/san_franci

test: Enable the model to do tool calling try to bind the tool

In [4]:
model_with_tools = model.bind_tools(tools)

response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []
ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_HxfFd94Wu3lIUqdhTSQIzfPv', 'type': 'tool_call'}]


We can see that there's now no text content, but there is a tool call! It wants us to call the Tavily Search tool.

This isn't calling that tool yet - it's just telling us to. In order to actually call it, we'll want to create our agent.

Create and Run the Agent

In [5]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools) # create_react_agent will call .bind_tools

response = agent_executor.invoke({
    "messages": [HumanMessage(content="hi!")]})

response["messages"]

response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='3575ceca-9ce8-4542-9adb-1e5ac0c40150'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_F8AYRi8BgAVEejMbULC3JFBs', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 88, 'total_tokens': 110, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1a9329a9-12e8-4661-a28e-674a60a42053-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_F8AYRi8BgAVEejMbULC3JFBs', 'type': 'too